# Basic CSGO Analysis
#### *Last Updated: August 8, 2021*
The csgo package was developed with easy analysis in mind. To that end, the data parsed goes directly into Pandas DataFrames, as shown in the first example notebook, [Parsing a CSGO demofile](https://github.com/pnxenopoulos/csgo/blob/master/examples/00_Parsing_a_CSGO_Demofile.ipynb). To efficiently calculate aggregate statistics from these Pandas Dataframes, the package contains `calc_stats()`, which filters, groups, and aggregates data based on user input. Furthermore, the package contains thirteen functions derived from `calc_stats()` to calculate standard CSGO aggregate statistics. 

To start, we reference the [demofile](https://www.hltv.org/matches/2349180/gambit-vs-natus-vincere-blast-premier-spring-final-2021) for a match between Gambit and Natus Vincere, where we look at the first map of the series, `de_dust2`.

In [1]:
import operator
from typing import Dict, List, Tuple, Union

import pandas as pd

from csgo.parser import DemoParser

# Create the parser object.
# Set log=True above if you want to produce a logfile for the parser.
demo_parser = DemoParser(demofile = "gambit-vs-natus-vincere-m1-dust2.dem", 
                         demo_id = "GA-NaVi-BLAST2021", parse_rate=128)

# Parse the demofile, output results to a dictionary and a dataframe.
data = demo_parser.parse()
data_df = demo_parser.parse(return_type="df")

01:44:13 [INFO] Go version>=1.14.0
01:44:13 [INFO] Initialized CSGODemoParser with demofile C:\Users\aagrawal-22\CSGO_Demofiles\gambit-vs-natus-vincere-m1-dust2.dem
01:44:13 [INFO] Setting demo id to GA-NaVi-BLAST2021
01:44:13 [INFO] Setting parse rate to 128
01:44:13 [INFO] Running Golang parser from C:\Users\aagrawal-22\Anaconda3\lib\site-packages\csgo-0.1-py3.8.egg\csgo\parser\
01:44:13 [INFO] Looking for file at C:\Users\aagrawal-22\CSGO_Demofiles\gambit-vs-natus-vincere-m1-dust2.dem
01:44:46 [INFO] Wrote demo parse output to GA-NaVi-BLAST2021.json
01:44:46 [INFO] Reading in JSON from GA-NaVi-BLAST2021.json
01:44:47 [INFO] JSON data loaded, available in the `json` attribute to parser
01:44:47 [INFO] Successfully parsed JSON output
01:44:47 [INFO] Successfully returned JSON output
01:44:47 [INFO] Running Golang parser from C:\Users\aagrawal-22\Anaconda3\lib\site-packages\csgo-0.1-py3.8.egg\csgo\parser\
01:44:47 [INFO] Looking for file at C:\Users\aagrawal-22\CSGO_Demofiles\gambit-vs

## `calc_stats()` 
`calc_stats()` can be used to calculate aggregate statistics from any of the Pandas DataFrames containing event data. It also allows the user to pass column filters. For example, we can use the function to calculate each player's headshot kills in the first half.

In [2]:
# Helper functions for calc_stats()
def extract_num_filters(filters: Dict[str, Union[List[bool], List[str]]], 
                        key: str) -> Tuple[List[str], List[float]]:
    sign_list = []
    val_list = []
    for index in filters[key]:
        if not isinstance(index, str):
            raise ValueError(f"Filter(s) for column \"{key}\" must be of type " 
                             f"string.")        
        i = 0 
        sign = ""
        while i < len(index) and not index[i].isdecimal(): 
            sign += index[i] 
            end_index = i 
            i += 1
        if sign not in ('==', '!=', '<=', '>=', '<', '>'): 
            raise Exception(f"Invalid logical operator in filters for \"{key}\""
                            f" column.") 
        sign_list.append(sign) 
        try:
            val_list.append(float(index[end_index + 1:])) 
        except ValueError as ve:
            raise Exception(f"Invalid numerical value in filters for \"{key}\" "
                            f"column.") from ve    
    return sign_list, val_list 

def check_filters(df: pd.DataFrame, 
                  filters: Dict[str, Union[List[bool], List[str]]]):
    for key in filters:
        if df.dtypes[key] == "bool":
            for index in filters[key]: 
                if not isinstance(index, bool): 
                    raise ValueError(f"Filter(s) for column \"{key}\" must be " 
                                     f"of type boolean")
        elif df.dtypes[key] == "O":
            for index in filters[key]: 
                if not isinstance(index, str): 
                    raise ValueError(f"Filter(s) for column \"{key}\" must be " 
                                     f"of type string")
        else:
            extract_num_filters(filters, key)  
            
def num_filter_df(df: pd.DataFrame,
                  col: str,
                  sign: str,
                  val: float) -> pd.DataFrame:
    ops = {"==":operator.eq(df[col], val), "!=":operator.ne(df[col], val),
           "<=":operator.le(df[col], val), ">=":operator.ge(df[col], val),
           "<":operator.lt(df[col], val), ">":operator.gt(df[col], val)}
    filtered_df = df.loc[ops[sign]]
    return filtered_df

def filter_df(df: pd.DataFrame,
              filters: Dict[str, Union[List[bool], List[str]]]) -> pd.DataFrame: 
    df_copy = df.copy() 
    check_filters(df_copy, filters) 
    for key in filters:
        if df_copy.dtypes[key] == 'bool' or df_copy.dtypes[key] == 'O': 
            df_copy = df_copy.loc[df_copy[key].isin(filters[key])]
        else:
            i = 0
            for sign in extract_num_filters(filters, key)[0]:
                val = extract_num_filters(filters, key)[1][i]
                df_copy = num_filter_df(df_copy, key, 
                                        extract_num_filters(filters, key)[0][i],
                                        val)
                i += 1
    return df_copy 

In [3]:
def calc_stats(df: pd.DataFrame, 
               filters: Dict[str, Union[List[bool], List[str]]], 
               col_to_groupby: List[str],
               col_to_agg: List[str],
               agg: List[List[str]],    
               col_names: List[str]) -> pd.DataFrame: 
    df_copy = filter_df(df, filters)
    agg_dict = dict(zip(col_to_agg, agg))
    if col_to_agg:
        df_copy = df_copy.groupby(col_to_groupby).agg(agg_dict).reset_index()
    df_copy.columns = col_names
    return df_copy

Below, the data is set to the `Kills` DataFrame, the data is filtered to where the value of the column `IsHeadshot` is True and the value of the column `RoundNum` is less than 16, the data is grouped by `AttackerName`, the column `AttackerName` is aggregated, the aggregation function `size()` is used, and the columns are renamed to `Player` and `1st Half HS`.

In [4]:
calc_stats(data_df["Kills"], {"IsHeadshot":[True], "RoundNum":["<=15"]},
           ["AttackerName"], ["AttackerName"], [["size"]], 
           ["Player", "1st Half HS"])

01:45:19 [INFO] NumExpr defaulting to 8 threads.


,Player,1st Half HS
0,Ax1Le,3
1,Boombl4,1
2,Hobbit,6
3,Perfecto,3
4,b1t,5
5,electronic,5
6,interz,5
7,nafany,2
8,s1mple,1


As mentioned earlier, the package contains thirteen functions derived from `calc_stats()` to efficiently calculate popular CSGO aggregate statistics. Unlike `calc_stats()`, the columns to group and aggregate the data by, the aggregation functions, and the column names do not need to be passed to these functions, only the data and column filters need to be passed.

# `accuracy()`
`accuracy()` takes in damage data, round data, and weapon fire data filters, and returns a DataFrame with weapon fires, accuracy, and headshot accuracy by player.

In [5]:
def accuracy(damage_data: pd.DataFrame,
             round_data_json: List[Dict],
             weapon_fires_filters: Dict[str, Union[List[bool], List[str]]] = {}
) -> pd.DataFrame: 
    weapon_fires = pd.DataFrame.from_dict(round_data_json[0]["WeaponFires"][0:])
    weapon_fires["RoundNum"] = 1
    for rd in round_data_json[1:]:
        rd_end = len(weapon_fires)
        weapon_fires = weapon_fires.append(pd.DataFrame.from_dict(
            rd["WeaponFires"][0:]))
        weapon_fires["RoundNum"][rd_end:] = rd["RoundNum"]              
    weapon_fires.reset_index(drop=True, inplace=True)
    weapon_fires = calc_stats(weapon_fires, weapon_fires_filters, ["PlayerName"], 
                              ["PlayerName"], [["size"]], ["Player", 
                                                           "Weapon Fires"])
    hits = calc_stats(damage_data, weapon_fires_filters, ["AttackerName"], 
                      ["AttackerName"], [["size"]], ["Player", "Hits"])
    headshots = calc_stats(damage_data.loc[damage_data["HitGroup"] == "Head"], 
                           weapon_fires_filters, ["AttackerName"], 
                           ["AttackerName"], [["size"]], ["Player", "Headshots"])
    acc = weapon_fires.merge(hits, how="outer").fillna(0)
    acc = acc.merge(headshots, how="outer").fillna(0)
    acc["ACC%"] = acc["Hits"] / acc["Weapon Fires"]
    acc["HS ACC%"] = acc["Headshots"] / acc["Weapon Fires"]
    acc = acc[["Player", "Weapon Fires", "ACC%", "HS ACC%"]]
    acc.sort_values(by="ACC%", ascending=False, inplace=True)
    acc.reset_index(drop=True, inplace=True)
    return acc

accuracy(data_df["Damages"], data["GameRounds"])

<ipython-input-5-383efdf1cf08>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weapon_fires["RoundNum"][rd_end:] = rd["RoundNum"]


,Player,Weapon Fires,ACC%,HS ACC%
0,electronic,327,0.223242,0.042813
1,s1mple,220,0.200000,0.036364
2,sh1ro,178,0.191011,0.016854
3,nafany,369,0.181572,0.027100
4,Boombl4,407,0.176904,0.014742
5,interz,404,0.175743,0.024752
6,Hobbit,511,0.162427,0.039139
7,Ax1Le,430,0.158140,0.023256
8,b1t,365,0.142466,0.035616
9,Perfecto,336,0.095238,0.023810


# `kast()`
`kast()` takes in kill data, a string representing the combination of KAST statistics to use, kill data filters and death data filters, and returns a DataFrame with KAST percentage and statistics by player.

In [6]:
def kast(kill_data: pd.DataFrame, 
         kast_string: str = "KAST", 
         kill_filters: Dict[str, Union[List[bool], List[str]]] = {},
         death_filters: Dict[str, Union[List[bool], List[str]]] = {},
) -> pd.DataFrame:
    columns = ["Player", f"{kast_string.upper()}%"]
    kast_counts = {}
    kast_rounds = {}
    for stat in kast_string.upper():
        columns.append(stat)
    killers = calc_stats(kill_data.loc[kill_data["AttackerTeam"] != 
                                       kill_data["VictimTeam"]], 
                         kill_filters, ["RoundNum"], ["AttackerName"], 
                         [["sum"]], ["RoundNum", "Killers"])
    victims = calc_stats(kill_data, kill_filters, ["RoundNum"], ["VictimName"], 
                         [["sum"]], ["RoundNum", "Victims"])
    assisters = calc_stats(kill_data.loc[(kill_data["AttackerTeam"] != 
                                          kill_data["VictimTeam"]) & 
                                         (kill_data["AssistedFlash"] == 
                                          False)].fillna(""), kill_filters, 
                           ["RoundNum"], ["AssisterName"], [["sum"]], 
                           ["RoundNum","Assisters"])
    traded = calc_stats(kill_data.loc[(kill_data["AttackerTeam"] != 
                                       kill_data["VictimTeam"]) & 
                                      (kill_data["IsTrade"] == 
                                       True)].fillna(""), kill_filters, 
                        ["RoundNum"], ["PlayerTradedName"], [["sum"]],
                        ["RoundNum", "Traded"])
    kast_data = killers.merge(assisters, how="outer").fillna("")
    kast_data = kast_data.merge(victims, how="outer").fillna("")
    kast_data = kast_data.merge(traded, how="outer").fillna("")
    for player in kill_data["AttackerName"].unique():
        kast_counts[player] = [[0, 0, 0, 0] for i in range(len(kast_data))]
        kast_rounds[player] = [0, 0, 0, 0, 0] 
    for rd in kast_data.index:
        for player in kast_counts:
            if "K" in  kast_string.upper():
                kast_counts[player][rd][0] = kast_data.iloc[rd]["Killers"].count(
                    player)
                kast_rounds[player][1] += kast_data.iloc[rd]["Killers"].count(
                    player)
            if "A" in kast_string.upper():
                kast_counts[player][rd][1] = kast_data.iloc[rd]["Assisters"].count(
                    player) 
                kast_rounds[player][2] += kast_data.iloc[rd]["Assisters"].count(
                    player)
            if "S" in kast_string.upper(): 
                if player not in kast_data.iloc[rd]["Victims"]:
                    kast_counts[player][rd][2] = 1 
                    kast_rounds[player][3] += 1
            if "T" in kast_string.upper():
                kast_counts[player][rd][3] = kast_data.iloc[rd]["Traded"].count(
                    player)  
                kast_rounds[player][4] += kast_data.iloc[rd]["Traded"].count(
                    player)
    for player in kast_rounds:
        for rd in kast_counts[player]:
            if any(rd):
                kast_rounds[player][0] += 1
        kast_rounds[player][0] /= len(kast_data)
    kast = pd.DataFrame.from_dict(kast_rounds, orient='index').reset_index()
    kast.fillna(0, inplace=True)
    kast.columns = ["Player", f"{kast_string.upper()}%", "K", "A", "S", "T"]
    kast = kast[columns]
    kast.sort_values(by=f"{kast_string.upper()}%", ascending=False, inplace=True)
    kast.reset_index(drop=True, inplace=True)
    return kast

kast(data_df["Kills"], "KAST")

,Player,KAST%,K,A,S,T
0,sh1ro,0.800000,14,5,16,2
1,s1mple,0.766667,22,4,16,2
2,interz,0.766667,14,1,15,1
3,electronic,0.733333,18,4,12,2
4,Perfecto,0.733333,10,2,14,4
5,Boombl4,0.700000,12,2,14,2
6,b1t,0.666667,23,1,12,1
7,Ax1Le,0.666667,18,2,14,3
8,Hobbit,0.633333,24,3,11,2
9,nafany,0.533333,11,3,9,2


# `kill_stats()`
`kill_stats()` takes in damage data, kill data, round data, and filters for each group of data, and returns a DataFrame with kills, deaths, assists, plus-minus, first kills, first kils plus-minus, trades, headshots, headshot percentage, accuracy, headshot accuracy, kill-death ratio, kills per round, and KAST percentage by player.

In [7]:
def kill_stats(damage_data: pd.DataFrame,
               kill_data: pd.DataFrame,
               round_data: pd.DataFrame,
               round_data_json: List[Dict],
               kill_filters: Dict[str, Union[List[bool], List[str]]] = {},
               death_filters: Dict[str, Union[List[bool], List[str]]] = {},
               round_filters: Dict[str, Union[List[bool], List[str]]] = {},
               weapon_fires_filters: Dict[str, Union[List[bool], 
                                                     List[str]]] = {}
) -> pd.DataFrame:
    kills = calc_stats(kill_data.loc[kill_data["AttackerTeam"] != 
                                     kill_data["VictimTeam"]],
                       kill_filters, ["AttackerName"], ["AttackerName"], 
                       [["size"]], ["Player", "K"])
    deaths = calc_stats(kill_data, death_filters, ["VictimName"], 
                        ["VictimName"], [["size"]], ["Player", "D"])
    assists = calc_stats(kill_data.loc[(kill_data["AttackerTeam"] != 
                                        kill_data["VictimTeam"]) & 
                                       (kill_data["AssistedFlash"] == False)],
                         kill_filters, ["AssisterName"], ["AssisterName"], 
                         [["size"]], ["Player", "A"])
    first_kills = calc_stats(kill_data.loc[(kill_data["AttackerTeam"] != 
                                            kill_data["VictimTeam"]) &
                                           (kill_data["IsFirstKill"] == True)],
                             kill_filters, ["AttackerName"], ["AttackerName"], 
                             [["size"]], ["Player", "FK"])
    first_deaths = calc_stats(kill_data.loc[(kill_data["AttackerTeam"] != 
                                            kill_data["VictimTeam"]) &
                                           (kill_data["IsFirstKill"] == True)],
                             kill_filters, ["VictimName"], ["VictimName"], 
                             [["size"]], ["Player", "FD"])
    headshots = calc_stats(kill_data.loc[(kill_data["AttackerTeam"] != 
                                          kill_data["VictimTeam"]) & 
                                         (kill_data["IsHeadshot"] == True)], 
                           kill_filters, ["AttackerName"], ["AttackerName"], 
                           [["size"]], ["Player", "HS"])
    headshot_pct = calc_stats(kill_data.loc[kill_data["AttackerTeam"] != 
                                            kill_data["VictimTeam"]], 
                              kill_filters, ["AttackerName"], ["IsHeadshot"], 
                              [["mean"]], ["Player", "HS%"])
    acc_stats = accuracy(damage_data, round_data_json, weapon_fires_filters)
    kast_stats = kast(kill_data, "KAST", kill_filters, death_filters)
    kill_stats = kills.merge(assists, how="outer").fillna(0)
    kill_stats = kill_stats.merge(deaths, how="outer").fillna(0)
    kill_stats = kill_stats.merge(first_kills, how="outer").fillna(0)
    kill_stats = kill_stats.merge(first_deaths, how="outer").fillna(0)
    kill_stats = kill_stats.merge(headshots, how="outer").fillna(0)
    kill_stats = kill_stats.merge(headshot_pct, how="outer").fillna(0)
    kill_stats = kill_stats.merge(acc_stats, how="outer").fillna(0)
    kill_stats = kill_stats.merge(kast_stats, how="outer").fillna(0)
    kill_stats["+/-"] = kill_stats["K"] - kill_stats["D"]
    kill_stats["KDR"] = kill_stats["K"] / kill_stats["D"]
    kill_stats["KPR"] = kill_stats["K"] / len(calc_stats(round_data, 
                                                         round_filters, [], [],                                                          
                                                         [], round_data.columns))
    kill_stats["FK +/-"] = kill_stats["FK"] - kill_stats["FD"]
    kill_stats[["K", "D", "A", "+/-", "FK", "FK +/-", "T", "HS"]] = kill_stats[[
        "K", "D", "A", "+/-", "FK", "FK +/-", "T", "HS"]].astype(int)
    kill_stats["HS%"] = kill_stats["HS%"].astype(float)
    kill_stats = kill_stats[["Player", "K", "D", "A", "+/-", "FK", "FK +/-", 
                             "T", "HS", "HS%", "ACC%", "HS ACC%", "KDR", "KPR", 
                             "KAST%"]]
    kill_stats.sort_values(by="K", ascending=False, inplace=True)
    kill_stats.reset_index(drop=True, inplace=True)
    return kill_stats

kill_stats(data_df["Damages"], data_df["Kills"], data_df["Rounds"], data["GameRounds"])

<ipython-input-5-383efdf1cf08>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weapon_fires["RoundNum"][rd_end:] = rd["RoundNum"]


,Player,K,D,A,+/-,FK,FK +/-,T,HS,HS%,ACC%,HS ACC%,KDR,KPR,KAST%
0,Hobbit,24,19,3,5,5,0,2,16,0.666667,0.162427,0.039139,1.263158,0.800000,0.633333
1,b1t,23,18,1,5,4,1,1,11,0.478261,0.142466,0.035616,1.277778,0.766667,0.666667
2,s1mple,22,14,4,8,4,3,2,6,0.272727,0.200000,0.036364,1.571429,0.733333,0.766667
3,Ax1Le,18,16,2,2,5,3,3,10,0.555556,0.158140,0.023256,1.125000,0.600000,0.666667
4,electronic,18,18,4,0,3,-2,2,9,0.500000,0.223242,0.042813,1.000000,0.600000,0.733333
5,interz,14,15,1,-1,1,-1,1,7,0.500000,0.175743,0.024752,0.933333,0.466667,0.766667
6,sh1ro,14,14,5,0,3,3,2,2,0.142857,0.191011,0.016854,1.000000,0.466667,0.800000
7,Boombl4,12,16,2,-4,1,-6,2,3,0.250000,0.176904,0.014742,0.750000,0.400000,0.700000
8,nafany,11,21,3,-10,3,-1,2,5,0.454545,0.181572,0.027100,0.523810,0.366667,0.533333
9,Perfecto,10,16,2,-6,1,0,4,7,0.700000,0.095238,0.023810,0.625000,0.333333,0.733333


# `adr()`
`adr()` takes in damage data, round data, and filters for each group of data, and returns a DataFrame with normalized and raw ADR by player.

In [8]:
def adr(damage_data: pd.DataFrame,
        round_data: pd.DataFrame,
        damage_filters: Dict[str, Union[List[bool], List[str]]] = {},
        round_filters: Dict[str, Union[List[bool], List[str]]] = {}
) -> pd.DataFrame:  
    adr = calc_stats(damage_data.loc[damage_data["AttackerTeam"] != 
                                     damage_data["VictimTeam"]], damage_filters, 
                     ["AttackerName"],["HpDamageTaken", "HpDamage"], [["sum"], 
                                                                      ["sum"]],
                     ["Player", "Norm ADR", "Raw ADR"])
    adr["Norm ADR"] = adr["Norm ADR"] / len(calc_stats(round_data, round_filters, 
                                                       [], [], [], 
                                                       round_data.columns))
    adr["Raw ADR"] = adr["Raw ADR"] / len(calc_stats(round_data, round_filters, 
                                                     [], [], [], 
                                                     round_data.columns))
    adr.sort_values(by="Norm ADR", ascending=False, inplace=True)
    adr.reset_index(drop=True, inplace=True)
    return adr

adr(data_df["Damages"], data_df["Rounds"])

,Player,Norm ADR,Raw ADR
0,s1mple,92.700000,115.833333
1,Hobbit,88.000000,119.333333
2,b1t,72.866667,138.966667
3,electronic,70.133333,85.866667
4,Ax1Le,58.633333,74.566667
5,Boombl4,57.100000,64.333333
6,sh1ro,56.833333,76.333333
7,interz,54.033333,63.733333
8,nafany,54.000000,62.033333
9,Perfecto,34.133333,48.033333


# `util_dmg()`
`util_dmg()` takes in damage data, grenade data, and filters for each group of data, and returns a DataFrame with given utility damage, utility damage, grenades thrown, given utility damage per grenade, and utility damage per grenade by player.

In [9]:
def util_dmg(damage_data: pd.DataFrame,
             grenade_data: pd.DataFrame,
             damage_filters: Dict[str, Union[List[bool], List[str]]] = {},
             grenade_filters: Dict[str, Union[List[bool], List[str]]] = {}
) -> pd.DataFrame:
    util_dmg = calc_stats(damage_data.loc[(damage_data["AttackerTeam"] != 
                                           damage_data["VictimTeam"]) & 
                                          (damage_data["Weapon"].isin([ 
                                               "HE Grenade", 
                                               "Incendiary Grenade", 
                                               "Molotov"
                                          ]))], damage_filters, ["AttackerName"], 
                          ["HpDamageTaken", "HpDamage"], [["sum"], ["sum"]], 
                          ["Player", "Given UD", "UD"])
    nades_thrown = calc_stats(grenade_data.loc[grenade_data["GrenadeType"].isin([
                                                   "HE Grenade", 
                                                   "Incendiary Grenade", 
                                                   "Molotov"
                                               ])], grenade_filters, 
                              ["PlayerName"], ["PlayerName"], [["size"]], 
                              ["Player", "Nades Thrown"])
    util_dmg_stats = util_dmg.merge(nades_thrown, how="outer").fillna(0)
    util_dmg_stats["Given UD Per Nade"] = (util_dmg_stats["Given UD"] 
                                           / util_dmg_stats["Nades Thrown"])
    util_dmg_stats["UD Per Nade"] = (util_dmg_stats["UD"] 
                                     / util_dmg_stats["Nades Thrown"])
    util_dmg_stats.sort_values(by="Given UD", ascending=False, inplace=True)
    util_dmg_stats.reset_index(drop=True, inplace=True)
    return util_dmg_stats

util_dmg(data_df["Damages"], data_df["Grenades"])

,Player,Given UD,UD,Nades Thrown,Given UD Per Nade,UD Per Nade
0,Boombl4,277,277,34,8.147059,8.147059
1,Hobbit,191,191,29,6.586207,6.586207
2,nafany,176,176,25,7.040000,7.040000
3,electronic,111,111,26,4.269231,4.269231
4,sh1ro,105,105,16,6.562500,6.562500
5,s1mple,66,70,10,6.600000,7.000000
6,Perfecto,48,48,35,1.371429,1.371429
7,Ax1Le,45,45,25,1.800000,1.800000
8,b1t,34,34,24,1.416667,1.416667
9,interz,27,27,32,0.843750,0.843750


# `flash_stats()`
`flash_stats()` takes in flash data, grenade data, and filters for each group of data, and returns a DataFrame with enemies flashed, team flashes, flashes thrown, and enemies flashed per throw by player.

In [10]:
def flash_stats(flash_data: pd.DataFrame,
                grenade_data: pd.DataFrame,
                flash_filters: Dict[str, Union[List[bool], List[str]]] = {},
                grenade_filters: Dict[str, Union[List[bool], List[str]]] = {},
) -> pd.DataFrame:    
    enemy_flashes = calc_stats(flash_data.loc[flash_data["AttackerTeam"] != 
                                              flash_data["PlayerTeam"]], 
                               flash_filters, ["AttackerName"], ["AttackerName"], 
                               [["size"]], ["Player", "EF"])
    team_flashes = calc_stats(flash_data.loc[flash_data["AttackerTeam"] == 
                                             flash_data["PlayerTeam"]], 
                              flash_filters, ["AttackerName"], ["AttackerName"], 
                              [["size"]], ["Player", "TF"])
    flashes_thrown = calc_stats(grenade_data.loc[grenade_data["GrenadeType"] == 
                                                 "Flashbang"], flash_filters, 
                                ["PlayerName"], ["PlayerName"], [["size"]], 
                                ["Player", "Flashes Thrown"])
    flash_stats = enemy_flashes.merge(team_flashes, how="outer").fillna(0)
    flash_stats = flash_stats.merge(flashes_thrown, how="outer").fillna(0)
    flash_stats["EF Per Throw"] = flash_stats["EF"] / flash_stats["Flashes Thrown"]
    flash_stats.sort_values(by="EF", ascending=False, inplace=True)
    flash_stats.reset_index(drop=True, inplace=True)
    return flash_stats

flash_stats(data_df["Flashes"], data_df["Grenades"])

,Player,EF,TF,Flashes Thrown,EF Per Throw
0,s1mple,55,52,36,1.527778
1,electronic,47,36,28,1.678571
2,b1t,40,35,30,1.333333
3,sh1ro,40,53,36,1.111111
4,Perfecto,36,48,27,1.333333
5,interz,36,37,24,1.500000
6,Ax1Le,35,27,32,1.093750
7,Boombl4,32,47,32,1.000000
8,Hobbit,28,20,23,1.217391
9,nafany,24,16,14,1.714286


# `bomb_stats()`
`bomb_stats()` takes in bomb data and bomb data filters, and returns a DataFrame with bomb plants, defuses, and defuse percentage by side and bombsite.

In [11]:
def bomb_stats(bomb_data: pd.DataFrame,
               bomb_filters: Dict[str, Union[List[bool], List[str]]] = {},
) -> pd.DataFrame:  
    team_one = bomb_data["PlayerTeam"].unique()[0]
    team_two = bomb_data["PlayerTeam"].unique()[1]
    team_one_plants = calc_stats(bomb_data.loc[(bomb_data["BombAction"] == 
                                                "plant") & 
                                               (bomb_data["PlayerTeam"] == 
                                                team_one)], bomb_filters, 
                                 ["BombSite"], ["BombSite"], [["size"]], 
                                 ["Bombsite", f"{team_one} Plants"])
    team_two_plants = calc_stats(bomb_data.loc[(bomb_data["BombAction"] == 
                                                "plant") & 
                                               (bomb_data["PlayerTeam"] == 
                                                team_two)], bomb_filters, 
                                 ["BombSite"], ["BombSite"], [["size"]],
                                 ["Bombsite", f"{team_two} Plants"])
    team_one_defuses = calc_stats(bomb_data.loc[(bomb_data["BombAction"] == 
                                                 "defuse") & 
                                                (bomb_data["PlayerTeam"] == 
                                                 team_one)], bomb_filters, 
                                  ["BombSite"], ["BombSite"], [["size"]],
                                  ["Bombsite", f"{team_one} Defuses"])
    team_two_defuses = calc_stats(bomb_data.loc[(bomb_data["BombAction"] == 
                                                 "defuse") & 
                                                (bomb_data["PlayerTeam"] == 
                                                 team_two)], bomb_filters, 
                                  ["BombSite"], ["BombSite"], [["size"]],
                                  ["Bombsite", f"{team_two} Defuses"])
    bomb_stats = team_one_plants.merge(team_two_defuses, 
                                       how="outer").fillna(0)
    bomb_stats[f"{team_two} Defuse %"] = (bomb_stats[f"{team_two} Defuses"] 
                                         / bomb_stats[f"{team_one} Plants"])
    bomb_stats = bomb_stats.merge(team_two_plants, how="outer").fillna(0)
    bomb_stats = bomb_stats.merge(team_one_defuses, how="outer").fillna(0)
    bomb_stats[f"{team_one} Defuse %"] = (bomb_stats[f"{team_one} Defuses"] 
                                         / bomb_stats[f"{team_two} Plants"])
    bomb_stats.loc[2]=["A and B", bomb_stats[f"{team_one} Plants"].sum(), 
                       bomb_stats[f"{team_two} Defuses"].sum(),
                       (bomb_stats[f"{team_two} Defuses"].sum() / 
                        bomb_stats[f"{team_one} Plants"].sum()), 
                       bomb_stats[f"{team_two} Plants"].sum(), 
                       bomb_stats[f"{team_one} Defuses"].sum(),
                       (bomb_stats[f"{team_one} Defuses"].sum() / 
                        bomb_stats[f"{team_two} Plants"].sum())] 
    bomb_stats.fillna(0, inplace=True)
    return bomb_stats

bomb_stats(data_df["BombEvents"])

,Bombsite,Gambit Plants,Natus Vincere Defuses,Natus Vincere Defuse %,Natus Vincere Plants,Gambit Defuses,Gambit Defuse %
0,A,6,1,0.166667,6,3,0.500000
1,B,3,1,0.333333,3,1,0.333333
2,A and B,9,2,0.222222,9,4,0.444444


# `econ_stats()`
`econ_stats()` takes in round data and round data filters and returns a DataFrame with buy type, average equipment value, average cash, and average spend by side.

In [12]:
def econ_stats(round_data: pd.DataFrame,
               round_data_json: List[Dict],
               round_filters: Dict[str, Union[List[bool], List[str]]] = {}
) -> pd.DataFrame:  
    team_one = round_data_json[0]["CTTeam"]
    team_one_CT_val = 0
    team_one_T_val = 0
    team_one_CT_cash = 0        
    team_one_T_cash = 0
    team_one_CT_spend = 0
    team_one_T_spend = 0
    team_two = round_data_json[0]["TTeam"]
    team_two_CT_val = 0
    team_two_T_val = 0
    team_two_CT_cash = 0
    team_two_T_cash = 0
    team_two_CT_spend = 0
    team_two_T_spend = 0
    first_half = 0
    second_half = 0
    filtered_round_data_json = []
    team_one_CT_buy = calc_stats(round_data.loc[round_data["RoundNum"] <= 15], 
                                 round_filters, ["CTBuyType"], ["CTBuyType"], 
                                 [["size"]], ["Side", f"{team_one} CT"])
    team_one_T_buy = calc_stats(round_data.loc[round_data["RoundNum"] > 15], 
                                round_filters, ["TBuyType"], ["TBuyType"], 
                                [["size"]], ["Side", f"{team_one} T"])
    team_two_CT_buy = calc_stats(round_data.loc[round_data["RoundNum"] > 15],
                                 round_filters, ["CTBuyType"], ["CTBuyType"], 
                                 [["size"]], ["Side", f"{team_two} CT"])
    team_two_T_buy = calc_stats(round_data.loc[round_data["RoundNum"] <= 15],
                                round_filters, ["TBuyType"], ["TBuyType"], 
                                [["size"]], ["Side", f"{team_two} T"])
    rounds = filter_df(round_data, round_filters)["RoundNum"].unique()
    for rd in rounds:
        filtered_round_data_json.append(round_data_json[rd-1])
    for rd in filtered_round_data_json:
        if rd["RoundNum"] <= 15:
            team_one_CT_val += rd["CTStartEqVal"]
            team_one_CT_cash += rd["CTRoundStartMoney"]
            team_one_CT_spend += rd["CTSpend"]
            team_two_T_val += rd["TStartEqVal"]
            team_two_T_cash += rd["TRoundStartMoney"]
            team_two_T_spend += rd["TSpend"]
            first_half += 1
        else:                                          
            team_one_T_val += rd["TStartEqVal"]
            team_one_T_cash += rd["TRoundStartMoney"]
            team_one_T_spend += rd["TSpend"]
            team_two_CT_val += rd["CTStartEqVal"]
            team_two_CT_cash += rd["CTRoundStartMoney"]
            team_two_CT_spend += rd["CTSpend"]
            second_half += 1
    if first_half == 0: first_half = 1
    if second_half == 0: second_half = 1
    team_one_CT_val /= first_half
    team_one_CT_cash /= first_half
    team_one_CT_spend /= first_half
    team_one_T_val /= second_half
    team_one_T_cash /= second_half
    team_one_T_spend /= second_half
    team_two_CT_val /= second_half
    team_two_CT_cash /= second_half
    team_two_CT_spend /= second_half
    team_two_T_val /= first_half
    team_two_T_cash /= first_half
    team_two_T_spend /= first_half
    econ_stats = team_one_CT_buy.merge(team_one_T_buy, how="outer").fillna(0)    
    econ_stats = econ_stats.merge(team_two_CT_buy, how="outer").fillna(0)  
    econ_stats = econ_stats.merge(team_two_T_buy, how="outer").fillna(0) 
    econ_stats.loc[len(econ_stats)] = ["Avg EQ Value", team_one_CT_val, 
                                       team_one_T_val, team_two_CT_val, 
                                       team_two_T_val]
    econ_stats.loc[len(econ_stats)] = ["Avg Cash", team_one_CT_cash, 
                                       team_one_T_cash, team_two_CT_cash, 
                                       team_two_T_cash]
    econ_stats.loc[len(econ_stats)] = ["Avg Spend", team_one_CT_spend, 
                                       team_one_T_spend, team_two_CT_spend, 
                                       team_two_T_spend]
    econ_stats.fillna(0, inplace=True)
    econ_stats.iloc[:, 1:] = econ_stats.iloc[:, 1:].astype(int)
    econ_stats = econ_stats.transpose()
    econ_stats.reset_index(inplace=True)
    econ_stats.columns = econ_stats.iloc[0]
    econ_stats.drop(0, inplace=True)
    econ_stats.reset_index(drop=True, inplace=True)
    return econ_stats


econ_stats(data_df["Rounds"], data["GameRounds"])

,Side,Eco,Full Buy,Half Buy,Pistol,Full Eco,Avg EQ Value,Avg Cash,Avg Spend
0,Natus Vincere CT,1,12,1,1,0,27386,29663,12713
1,Natus Vincere T,0,11,3,1,0,22133,25180,13250
2,Gambit CT,0,13,0,1,1,24183,20523,13403
3,Gambit T,1,12,1,1,0,21023,19510,12946


# `kill_breakdown()`
`kill_breakdown()` takes in kill data and kill data filters, and returns a DataFrame with kills by weapon type by player.

In [13]:
# Helper function for kill_breakdown()
def weapon_type(weapon: str) -> str:
    if weapon in ["Knife"]:
        return "Melee Kills"
    elif weapon in ["CZ-75 Auto", "Desert Eagle", "Dual Berettas", "Five-SeveN",
                    "Glock-18", "P2000", "P250", "R8 Revolver", "Tec-9", 
                    "USP-S"]:
        return "Pistol Kills"
    elif weapon in ["MAG-7", "Nova", "Sawed-Off", "XM1014"]:
        return "Shotgun Kills"
    elif weapon in ["MAC-10", "MP5-SD", "MP7", "MP9", "P90", "PP-Bizon",
                    "UMP-45"]:
        return "SMG Kills"
    elif weapon in ["AK-47", "AUG", "FAMAS", "Galil AR", "M4A1-S", "M4A4",
                    "SG 553"]:
        return "Assault Rifle Kills"
    elif weapon in ["M249", "Negev"]:
        return "Machine Gun Kills"
    elif weapon in ["AWP", "G3SG1", "SCAR-20", "SSG 08"]:
        return "Sniper Rifle Kills"
    else:
        return "Utility Kills"
    
    
def kill_breakdown(kill_data: pd.DataFrame,
                   kill_filters: Dict[str, Union[List[bool], List[str]]] = {}
) -> pd.DataFrame:
    kill_breakdown = kill_data.loc[kill_data["AttackerTeam"] != 
                                   kill_data["VictimTeam"]].copy()
    kill_breakdown["Kills Type"] = kill_breakdown.apply(lambda row: weapon_type(
                                                            row["Weapon"]), 
                                                        axis=1)
    kill_breakdown = calc_stats(kill_breakdown, kill_filters, ["AttackerName", 
                                                               "Kills Type"],
                                ["AttackerName"], [["size"]], [
                                                               "Player", 
                                                               "Kills Type", 
                                                               "Kills"
                                ])
    kill_breakdown = kill_breakdown.pivot(index="Player", columns="Kills Type",
                                         values="Kills")
    for col in ["Melee Kills", "Pistol Kills", "Shotgun Kills", "SMG Kills", 
                "Assault Rifle Kills", "Machine Gun Kills", "Sniper Rifle Kills", 
                "Utility Kills"]:
        if not col in kill_breakdown.columns:
            kill_breakdown.insert(0, col, 0)
        kill_breakdown[col].fillna(0, inplace=True)
        kill_breakdown[col] = kill_breakdown[col].astype(int)
    kill_breakdown["Total Kills"] = kill_breakdown.iloc[0:].sum(axis=1)
    kill_breakdown.reset_index(inplace=True)
    kill_breakdown = kill_breakdown.rename_axis(None, axis=1)
    kill_breakdown = kill_breakdown[["Player", "Melee Kills", "Pistol Kills",
                                     "Shotgun Kills", "SMG Kills", 
                                     "Assault Rifle Kills", "Machine Gun Kills",
                                     "Sniper Rifle Kills", "Utility Kills", 
                                     "Total Kills"]]
    kill_breakdown.sort_values(by="Total Kills", ascending=False, inplace=True)
    kill_breakdown.reset_index(drop=True, inplace=True)
    return kill_breakdown

kill_breakdown(data_df["Kills"])

,Player,Melee Kills,Pistol Kills,Shotgun Kills,SMG Kills,Assault Rifle Kills,Machine Gun Kills,Sniper Rifle Kills,Utility Kills,Total Kills
0,Hobbit,0,4,0,1,19,0,0,0,24
1,b1t,0,2,0,1,8,0,12,0,23
2,s1mple,0,3,0,0,5,0,13,1,22
3,Ax1Le,0,4,0,1,13,0,0,0,18
4,electronic,0,4,0,0,14,0,0,0,18
5,interz,0,0,0,2,12,0,0,0,14
6,sh1ro,0,1,0,0,0,0,11,2,14
7,Boombl4,0,0,0,0,10,0,2,0,12
8,nafany,0,4,0,0,7,0,0,0,11
9,Perfecto,0,2,0,0,8,0,0,0,10


# `util_dmg_breakdown()`
`util_dmg_breakdown()` takes in damage data, grenade data, and filters for each group of data, and returns a DataFrame with given utility damage, utility damage, grenades thrown, given utility damage per grenade, and utility damage per grenade by player and grenade type.

In [14]:
def util_dmg_breakdown(damage_data: pd.DataFrame,
                       grenade_data: pd.DataFrame,
                       damage_filters: Dict[str, Union[List[bool], 
                                                       List[str]]] = {},
                       grenade_filters: Dict[str, Union[List[bool], 
                                                        List[str]]] = {}
) -> pd.DataFrame:  
    util_dmg = calc_stats(damage_data.loc[(damage_data["AttackerTeam"] != 
                                           damage_data["VictimTeam"]) & 
                                          (damage_data["Weapon"].isin([
                                               "HE Grenade", 
                                               "Incendiary Grenade",
                                               "Molotov"
                                          ]))], damage_filters, ["AttackerName", 
                                                                 "Weapon"], 
                          ["HpDamageTaken", "HpDamage"], [["sum"], ["sum"]], 
                          ["Player", "Nade Type", "Given UD", "UD"])
    nades_thrown = calc_stats(grenade_data.loc[grenade_data["GrenadeType"].isin([
                                                   "HE Grenade", 
                                                   "Incendiary Grenade", 
                                                   "Molotov"
                                               ])], grenade_filters, 
                              ["PlayerName", "GrenadeType"], ["PlayerName"], 
                              [["size"]], ["Player", "Nade Type","Nades Thrown"])
    util_dmg_breakdown = util_dmg.merge(nades_thrown, how="outer", on = 
                                        ["Player", "Nade Type"]).fillna(0)
    util_dmg_breakdown["Given UD Per Nade"] = (util_dmg_breakdown["Given UD"]
                                               / util_dmg_breakdown["Nades Thrown"])
    util_dmg_breakdown["UD Per Nade"] = (util_dmg_breakdown["UD"] 
                                         / util_dmg_breakdown["Nades Thrown"])
    util_dmg_breakdown.sort_values(by=["Player", "Given UD"], ascending=[True, False], 
                                   inplace=True)
    util_dmg_breakdown.reset_index(drop=True, inplace=True)
    return util_dmg_breakdown

util_dmg_breakdown(data_df["Damages"], data_df["Grenades"])

,Player,Nade Type,Given UD,UD,Nades Thrown,Given UD Per Nade,UD Per Nade
0,Ax1Le,HE Grenade,26.0,26.0,9,2.888889,2.888889
1,Ax1Le,Molotov,19.0,19.0,6,3.166667,3.166667
2,Ax1Le,Incendiary Grenade,0.0,0.0,10,0.000000,0.000000
3,Boombl4,HE Grenade,213.0,213.0,13,16.384615,16.384615
4,Boombl4,Incendiary Grenade,40.0,40.0,11,3.636364,3.636364
5,Boombl4,Molotov,24.0,24.0,10,2.400000,2.400000
6,Hobbit,HE Grenade,167.0,167.0,10,16.700000,16.700000
7,Hobbit,Molotov,24.0,24.0,9,2.666667,2.666667
8,Hobbit,Incendiary Grenade,0.0,0.0,10,0.000000,0.000000
9,Perfecto,HE Grenade,48.0,48.0,15,3.200000,3.200000


# `win_breakdown()`
`win_breakdown()` takes in round data and round data filters, and returns a DataFrame with win type by team.

In [15]:
def win_breakdown(round_data: pd.DataFrame,
                  round_filters: Dict[str, Union[List[bool], List[str]]] = {}
) -> pd.DataFrame:     
    round_data_copy = round_data.copy()
    round_data_copy.replace("BombDefused", "CT Bomb Defusal Wins", inplace=True)
    round_data_copy.replace("CTWin", "CT T Elim Wins", inplace=True)
    round_data_copy.replace("TargetBombed", "T Bomb Detonation Wins", inplace=True)
    round_data_copy.replace("TargetSaved", "CT Time Expired Wins", inplace=True)
    round_data_copy.replace("TerroristsWin", "T CT Elim Wins", inplace=True)
    win_breakdown = calc_stats(round_data_copy, round_filters, ["WinningTeam", 
                                                           "RoundEndReason"],
                               ["RoundEndReason"], [["size"]], [
                                                                "Team", 
                                                                "RoundEndReason", 
                                                                "Count"
                               ])
    win_breakdown = win_breakdown.pivot(index="Team", columns="RoundEndReason", 
                                        values="Count").fillna(0)
    win_breakdown.reset_index(inplace=True)
    win_breakdown = win_breakdown.rename_axis(None, axis=1)
    win_breakdown["Total CT Wins"] = win_breakdown.iloc[0:][list(
        set.intersection(set(win_breakdown.columns), 
                         set(["CT Bomb Defusal Wins", "CT T Elim Wins", 
                              "CT Time Expired Wins"])))].sum(axis=1)
    win_breakdown["Total T Wins"] =  win_breakdown.iloc[0:][list(
        set.intersection(set(win_breakdown.columns), 
                         set(["T Bomb Detonation Wins", "T CT Elim Wins"])))
        ].sum(axis=1)
    win_breakdown["Total Wins"] = win_breakdown.iloc[0:, 0:-2].sum(axis=1)
    win_breakdown.iloc[:, 1:] = win_breakdown.iloc[:, 1:].astype(int)
    return win_breakdown

win_breakdown(data_df["Rounds"])

,Team,CT Bomb Defusal Wins,CT T Elim Wins,CT Time Expired Wins,T Bomb Detonation Wins,T CT Elim Wins,Total CT Wins,Total T Wins,Total Wins
0,Gambit,4,1,4,4,3,9,7,16
1,Natus Vincere,2,2,4,4,2,8,6,14


# `player_box_score()`
`player_box_score()` takes in damage data, flash data, grenade data, kill data, round data, and filters for each group of data, and returns a player box score DataFrame containing statistics from each group of data.

In [16]:
def player_box_score(damage_data: pd.DataFrame,
                     flash_data: pd.DataFrame,
                     grenade_data: pd.DataFrame,
                     kill_data: pd.DataFrame,
                     round_data: pd.DataFrame,
                     round_data_json: List[Dict],
                     damage_filters: Dict[str, Union[List[bool], 
                                                     List[str]]] = {},
                     flash_filters: Dict[str, Union[List[bool], 
                                                    List[str]]] = {},
                     grenade_filters: Dict[str, Union[List[bool], 
                                                      List[str]]] = {},
                     kill_filters: Dict[str, Union[List[bool], List[str]]] = {},
                     death_filters: Dict[str, Union[List[bool], 
                                                    List[str]]] = {},
                     round_filters: Dict[str, Union[List[bool], List[str]]] = {},
                     weapon_fires_filters: Dict[str, Union[List[bool], 
                                                           List[str]]] = {}
) -> pd.DataFrame:
    k_stats = kill_stats(damage_data, kill_data, round_data, round_data_json, 
                         kill_filters, death_filters, round_filters, 
                         weapon_fires_filters)
    k_stats = k_stats[["Player", "K", "D", "A", "HS%", "ACC%", "HS ACC%", "KDR", 
                       "KAST%"]]
    adr_stats = adr(damage_data, round_data, damage_filters, round_filters)
    adr_stats = adr_stats[["Player", "Norm ADR"]]
    adr_stats.columns = ["Player", "ADR"]
    ud_stats = util_dmg(damage_data, grenade_data, damage_filters, grenade_filters)
    ud_stats = ud_stats[["Player", "UD", "UD Per Nade"]]
    f_stats = flash_stats(flash_data, grenade_data, flash_filters, 
                          grenade_filters)
    f_stats = f_stats[["Player", "EF", "EF Per Throw"]]
    box_score = k_stats.merge(adr_stats, how="outer").fillna(0)
    box_score = box_score.merge(ud_stats, how="outer").fillna(0)
    box_score = box_score.merge(f_stats, how="outer").fillna(0)
    return box_score

player_box_score(data_df["Damages"], data_df["Flashes"], data_df["Grenades"], 
                 data_df["Kills"], data_df["Rounds"], data["GameRounds"])

<ipython-input-5-383efdf1cf08>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weapon_fires["RoundNum"][rd_end:] = rd["RoundNum"]


,Player,K,D,A,HS%,ACC%,HS ACC%,KDR,KAST%,ADR,UD,UD Per Nade,EF,EF Per Throw
0,Hobbit,24,19,3,0.666667,0.162427,0.039139,1.263158,0.633333,88.000000,191,6.586207,28,1.217391
1,b1t,23,18,1,0.478261,0.142466,0.035616,1.277778,0.666667,72.866667,34,1.416667,40,1.333333
2,s1mple,22,14,4,0.272727,0.200000,0.036364,1.571429,0.766667,92.700000,70,7.000000,55,1.527778
3,Ax1Le,18,16,2,0.555556,0.158140,0.023256,1.125000,0.666667,58.633333,45,1.800000,35,1.093750
4,electronic,18,18,4,0.500000,0.223242,0.042813,1.000000,0.733333,70.133333,111,4.269231,47,1.678571
5,interz,14,15,1,0.500000,0.175743,0.024752,0.933333,0.766667,54.033333,27,0.843750,36,1.500000
6,sh1ro,14,14,5,0.142857,0.191011,0.016854,1.000000,0.800000,56.833333,105,6.562500,40,1.111111
7,Boombl4,12,16,2,0.250000,0.176904,0.014742,0.750000,0.700000,57.100000,277,8.147059,32,1.000000
8,nafany,11,21,3,0.454545,0.181572,0.027100,0.523810,0.533333,54.000000,176,7.040000,24,1.714286
9,Perfecto,10,16,2,0.700000,0.095238,0.023810,0.625000,0.733333,34.133333,48,1.371429,36,1.333333


# `team_box_score()`
`team_box_score()` takes in damage data, flash data, grenade data, kill data, round data, and filters for each group of data, and returns a team box score DataFrame containing statistics from each group of data.

In [17]:
def team_box_score(damage_data: pd.DataFrame,
                   flash_data: pd.DataFrame,
                   grenade_data: pd.DataFrame,
                   kill_data: pd.DataFrame,
                   round_data: pd.DataFrame,
                   round_data_json: List[Dict],
                   damage_filters: Dict[str, Union[List[bool], List[str]]] = {},
                   flash_filters: Dict[str, Union[List[bool], List[str]]] = {},
                   grenade_filters: Dict[str, Union[List[bool], List[str]]] = {},
                   kill_filters: Dict[str, Union[List[bool], List[str]]] = {},
                   death_filters: Dict[str, Union[List[bool], List[str]]] = {},
                   round_filters: Dict[str, Union[List[bool], List[str]]] = {},
                   weapon_fires_filters: Dict[str, Union[List[bool], 
                                                         List[str]]] = {}
) -> pd.DataFrame:
    kills = calc_stats(kill_data.loc[kill_data["AttackerTeam"] != 
                                        kill_data["VictimTeam"]], kill_filters, 
                          ["AttackerTeam"], ["AttackerTeam"], [["size"]], 
                          ["Team", "K"])
    deaths = calc_stats(kill_data, death_filters, ["VictimTeam"], ["VictimTeam"], 
                        [["size"]], ["Team", "D"])
    assists = calc_stats(kill_data.loc[(kill_data["AttackerTeam"] != 
                                        kill_data["VictimTeam"]) & 
                                       (kill_data["AssistedFlash"] == False)],
                         kill_filters, ["AssisterTeam"], ["AssisterTeam"], 
                         [["size"]], ["Team", "A"])
    first_kills = calc_stats(kill_data.loc[(kill_data["AttackerTeam"] != 
                                            kill_data["VictimTeam"]) &
                                           (kill_data["IsFirstKill"] == True)],
                             kill_filters, ["AttackerTeam"], ["AttackerTeam"], 
                             [["size"]], ["Team", "FK"])
    adr = calc_stats(damage_data.loc[damage_data["AttackerTeam"] != 
                                     damage_data["VictimTeam"]], 
                     damage_filters, ["AttackerTeam"],["HpDamageTaken"], 
                     [["sum"]], ["Team", "ADR"])
    adr["ADR"] = adr["ADR"] / len(calc_stats(round_data, round_filters, [], [], 
                                             [], round_data.columns))
    headshot_pct = calc_stats(kill_data.loc[kill_data["AttackerTeam"] != 
                                            kill_data["VictimTeam"]], 
                              kill_filters, ["AttackerTeam"], ["IsHeadshot"], 
                              [["mean"]], ["Team", "HS%"])
    weapon_fires = pd.DataFrame.from_dict(round_data_json[0]["WeaponFires"][0:])
    weapon_fires["RoundNum"] = 1
    for rd in round_data_json[1:]:
        rd_end = len(weapon_fires)
        weapon_fires = weapon_fires.append(pd.DataFrame.from_dict(
            rd["WeaponFires"][0:]))
        weapon_fires["RoundNum"][rd_end:] = rd["RoundNum"]              
    weapon_fires.reset_index(drop=True, inplace=True)
    weapon_fires = calc_stats(weapon_fires, weapon_fires_filters, ["PlayerTeam"], 
                              ["PlayerTeam"], [["size"]], ["Team", 
                                                           "Weapon Fires"])
    hits = calc_stats(damage_data, weapon_fires_filters, ["AttackerTeam"], 
                      ["AttackerTeam"], [["size"]], ["Team", "Hits"])
    headshots = calc_stats(damage_data.loc[damage_data["HitGroup"] == "Head"], 
                           weapon_fires_filters, ["AttackerTeam"], 
                           ["AttackerTeam"], [["size"]], ["Team", "Headshots"])
    acc = weapon_fires.merge(hits, how="outer").fillna(0)
    acc = acc.merge(headshots, how="outer").fillna(0)
    acc["ACC%"] = acc["Hits"] / acc["Weapon Fires"]
    acc["HS ACC%"] = acc["Headshots"] / acc["Weapon Fires"]
    acc = acc[["Team", "ACC%", "HS ACC%"]]
    util_dmg = calc_stats(damage_data.loc[(damage_data["AttackerTeam"] != 
                                           damage_data["VictimTeam"]) & 
                                          (damage_data["Weapon"].isin([ 
                                               "HE Grenade", 
                                               "Incendiary Grenade", 
                                               "Molotov"
                                          ]))], damage_filters, ["AttackerTeam"], 
                          ["HpDamage"], [["sum"]], ["Team", "UD"])
    nades_thrown = calc_stats(grenade_data.loc[grenade_data["GrenadeType"].isin([
                                                   "HE Grenade", 
                                                   "Incendiary Grenade", 
                                                   "Molotov"
                                               ])], grenade_filters, 
                              ["PlayerTeam"], ["PlayerTeam"], [["size"]], 
                              ["Team", "Nades Thrown"])   
    enemy_flashes = calc_stats(flash_data.loc[flash_data["AttackerTeam"] != 
                                              flash_data["PlayerTeam"]], 
                               flash_filters, ["AttackerTeam"], ["AttackerTeam"], 
                               [["size"]], ["Team", "EF"])
    flashes_thrown = calc_stats(grenade_data.loc[grenade_data["GrenadeType"] == 
                                                 "Flashbang"], flash_filters, 
                                ["PlayerTeam"], ["PlayerTeam"], [["size"]], 
                                ["Team", "Flashes Thrown"])
    econ = econ_stats(round_data, round_data_json, round_filters)
    team_one = round_data_json[0]["CTTeam"]
    box_score = kills.merge(deaths, how="outer").fillna(0)
    box_score = box_score.merge(assists, how="outer").fillna(0)
    box_score["+/-"] = box_score["K"] - box_score["D"]
    box_score = box_score.merge(first_kills, how="outer").fillna(0)
    box_score = box_score.merge(adr, how="outer").fillna(0)
    box_score = box_score.merge(headshot_pct, how="outer").fillna(0)
    box_score = box_score.merge(acc, how="outer").fillna(0)
    box_score = box_score.merge(util_dmg, how="outer").fillna(0)
    box_score = box_score.merge(nades_thrown, how="outer").fillna(0)
    box_score["UD Per Nade"] = box_score["UD"]  / box_score["Nades Thrown"]
    box_score = box_score.merge(enemy_flashes, how="outer").fillna(0)
    box_score = box_score.merge(flashes_thrown, how="outer").fillna(0)
    box_score["EF Per Throw"] = box_score["EF"] / box_score["Flashes Thrown"]      
    if box_score.iloc[0]["Team"] == team_one:
        for buy_type in econ.columns[1:-3]:
            box_score[buy_type] = [econ.iloc[0:2][buy_type].sum(), 
                                   econ.iloc[2:][buy_type].sum()]
        box_score["Avg EQ Value"] = [int(econ.iloc[0:2]["Avg EQ Value"].mean()), 
                                     int(econ.iloc[2:]["Avg EQ Value"].mean())] 
        box_score["Avg Cash"] = [int(econ.iloc[0:2]["Avg Cash"].mean()), 
                                 int(econ.iloc[2:]["Avg Cash"].mean())] 
        box_score["Avg Spend"] = [int(econ.iloc[0:2]["Avg Spend"].mean()), 
                                  int(econ.iloc[2:]["Avg Spend"].mean())] 
    else:
        for buy_type in econ.columns[1:-3]:
            box_score[buy_type] = [econ.iloc[2:][buy_type].sum(), 
                                   econ.iloc[0:2][buy_type].sum()]
        box_score["Avg EQ Value"] = [int(econ.iloc[2:]["Avg EQ Value"].mean()), 
                                     int(econ.iloc[0:2]["Avg EQ Value"].mean())] 
        box_score["Avg Cash"] = [int(econ.iloc[2:]["Avg Cash"].mean()), 
                                 int(econ.iloc[0:2]["Avg Cash"].mean())] 
        box_score["Avg Spend"] = [int(econ.iloc[2:]["Avg Spend"].mean()), 
                                  int(econ.iloc[0:2]["Avg Spend"].mean())] 
    box_score = box_score.merge(win_breakdown(round_data), how="outer").fillna(0)
    box_score.rename(columns={"Total CT Wins":"CT Wins", "Total T Wins":"T Wins", 
                              "Total Wins":"Score"}, inplace=True)
    score = box_score["Score"]
    ct_wins = box_score["CT Wins"]
    t_wins = box_score["T Wins"]
    box_score.drop(["Score", "CT Wins", "T Wins"], axis=1, inplace=True)
    box_score.insert(1, "Score", score)
    box_score.insert(2, "CT Wins", ct_wins)
    box_score.insert(3, "T Wins", t_wins)
    box_score = box_score.transpose()
    box_score.columns = box_score.iloc[0]
    box_score.drop("Team", inplace=True)
    box_score = box_score.rename_axis(None, axis=1)
    return box_score

team_box_score(data_df["Damages"], data_df["Flashes"], data_df["Grenades"], 
               data_df["Kills"], data_df["Rounds"], data["GameRounds"])

<ipython-input-17-881262c85309>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weapon_fires["RoundNum"][rd_end:] = rd["RoundNum"]


,Gambit,Natus Vincere
Score,16,14
CT Wins,9,8
T Wins,7,6
K,81,85
D,85,82
A,14,13
+/-,-4,3
FK,17,13
ADR,311.5,326.933333
HS%,0.493827,0.423529
